In [1]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
data = train.copy()
test = data_test.copy()
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
data['Age'][0]

22.0

###### age, cabin, embarked 비어있는 부분 확인

passengerld = 승객 id  
survived = 살아남았는가  
Pclass = ticket class(좌석 등급)  
name = 이름  
sex = 성별  
age = 나이  
SibSp = 타이타닉호에 탑승한 형제/배우자 수  
Parch = 타이타닉호에 탑승한 부모/자녀 수  
Ticket = 티켓 번호  
Fare = 요금  
Cabin = 선실번호 / 케비넷 번호?  
Embarked = 승선항 C = 셰르부르, Q = 퀸스타운, S = 사우샘프턴

In [5]:
to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
drop_data = data.copy()
drop_data = data.drop(to_drop, axis = 1)
test = test.drop(to_drop, axis = 1)
drop_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [6]:
def data_scan(data):
    #비어있는 Embarked 채우기
    from sklearn.impute import SimpleImputer
    Impute_freq = SimpleImputer(strategy='most_frequent')
    data['Embarked'] = Impute_freq.fit_transform(data[['Embarked']])
    
    #OneHotEncoder
    data = pd.get_dummies(data = data, columns = ['Embarked'], prefix = 'Embarked')
    data = pd.get_dummies(data = data, columns = ['Sex'])
    
    #Age -> 그룹을 나누어 비어있는 Age 채우기
    check1 = data['SibSp'].isin([0]) & data['Parch'].isin([0])
    check2 = (data.SibSp == 0) & (data.Parch != 0)
    check3 = (data.SibSp != 0) & (data.Parch == 0)
    check4 = (data.SibSp != 0) & (data.Parch != 0)
    data1 = data[check1]
    data2 = data[check2]
    data3 = data[check3]
    data4 = data[check4]

    for i in range(0, len(data)):
        if data['Age'][i] == 'NaN':
            if data[check1][i] == True:
                data['Age'][i] == data1['Age'].mean()
            elif data[check2][i] == True:
                data['Age'][i] == data2['Age'].mean()
            elif data[check3][i] == True:
                data['Age'][i] == data3['Age'].mean()
            elif data[check4][i] == True:
                data['Age'][i] == data4['Age'].mean()
    
    
    
    #스케일링
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    
    num_pipeline = Pipeline([
        ('Imputer', SimpleImputer(strategy = 'median')),
        ('std_scaler', StandardScaler())
    ])
    
    data = num_pipeline.fit_transform(data)
    
    return data

In [7]:
dataset = drop_data.copy()
X_train = dataset.drop(['Survived'], axis = 1)
y_train = dataset['Survived']
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S
887,1,female,19.0,0,0,30.0000,S
888,3,female,NaN,1,2,23.4500,S
889,1,male,26.0,0,0,30.0000,C


In [8]:
X_train = data_scan(X_train)
test = data_scan(test)
X_train

array([[ 0.82737724, -0.56573646,  0.43279337, ...,  0.61583843,
        -0.73769513,  0.73769513],
       [-1.56610693,  0.66386103,  0.43279337, ..., -1.62380254,
         1.35557354, -1.35557354],
       [ 0.82737724, -0.25833709, -0.4745452 , ...,  0.61583843,
         1.35557354, -1.35557354],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337, ...,  0.61583843,
         1.35557354, -1.35557354],
       [-1.56610693, -0.25833709, -0.4745452 , ..., -1.62380254,
        -0.73769513,  0.73769513],
       [ 0.82737724,  0.20276197, -0.4745452 , ..., -1.62380254,
        -0.73769513,  0.73769513]])

In [9]:
test

array([[ 0.87348191,  0.38623105, -0.49947002, ..., -1.35067551,
        -0.75592895,  0.75592895],
       [ 0.87348191,  1.37137004,  0.61699237, ...,  0.74037028,
         1.32287566, -1.32287566],
       [-0.31581919,  2.55353683, -0.49947002, ..., -1.35067551,
        -0.75592895,  0.75592895],
       ...,
       [ 0.87348191,  0.70147553, -0.49947002, ...,  0.74037028,
        -0.75592895,  0.75592895],
       [ 0.87348191, -0.20485235, -0.49947002, ...,  0.74037028,
        -0.75592895,  0.75592895],
       [ 0.87348191, -0.20485235,  0.61699237, ..., -1.35067551,
        -0.75592895,  0.75592895]])

In [10]:
y_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

In [17]:
def make_model(dense, dr_rate):
    from functools import partial
    DefaultDense = partial(keras.layers.Dense, kernel_initalizer = 'lecun_normal', activation = 'selu')
    
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape = [len(X_train[0]),]))
    model.add(DefaultDense(dense))
    model.add(DefaultDense(dense))
    model.add(DefaultDense(dense))
    model.add(keras.layers.Dropout(dr_rate))
    
    model.add(keras.layers.Dense(2, activation = 'softmax'))
    
    model.compile(loss = 'sparse_categorical_crossentropy',
                 optimizer = 'adam',
                 metrics = ['accuracy'])
    
    return model

In [18]:
param_grid = {
    'dense' : range(1, 100),
    'dr_rate' : range(0, 0.5, 0.1)
}
n_iter = 80

model = make_model(param_grid.de)

0.8440074906367039

In [19]:
predictions = vot_clf.predict(test)

In [20]:
predictions

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [21]:
output = pd.DataFrame({'PassengerId': data_test.PassengerId, 'Survived': predictions})
output.to_csv('Voting.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
